In [1]:
import torch
import sys

sys.path.append('../src')


from models.U_NET import UNet, BCEDiceLoss
from utils.data_loading import get_dataloaders
from utils.avaliation import compute_iou, compute_metrics, visualize_predictions

# Carregando os dados

In [2]:
# Obter os DataLoaders
train_loader, val_loader, test_loader, train_dataset, val_dataset = get_dataloaders(batch_size=32, num_workers=4)

# U-Net

In [3]:
import torch.optim as optim

model = UNet(in_channels=3, out_channels=1)
criterion = BCEDiceLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

UNet(
  (enc_conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (enc_conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (enc_conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affin

In [ ]:
num_epochs = 1

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        #outputs = outputs.squeeze(1)  # Ajustar dimensões
        
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
    
    # Validação
    model.eval()
    val_loss = 0.0
    total_iou = 0.0
    with torch.no_grad():
        for images, masks in val_loader:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            #outputs = outputs.squeeze(1) // manter a mesma proporção da mask [32, 1, 384, 384] com squeeze fica [32, 384, 384]
            
            loss = criterion(outputs, masks)
            val_loss += loss.item() * images.size(0)
            
            # Calcular IoU
            preds = torch.sigmoid(outputs)
            preds = (preds > 0.5).float()
            iou = compute_iou(preds, masks)
            total_iou += iou * images.size(0)
    
    val_loss /= len(val_dataset)
    avg_iou = total_iou / len(val_dataset)
    print(f"Validation Loss: {val_loss:.4f}, IoU: {avg_iou:.4f}")
    
    # Salvar o modelo
    torch.save(model.state_dict(), f'unet_epoch_{epoch+1}.pth')